In [5]:
from datasets import load_dataset
from config import OPENAI_API_KEY
import os
import openai
from tqdm import tqdm
import csv
import re

In [2]:
# load sagnikrayc/mctest
train_set = load_dataset('sagnikrayc/mctest', 'mc160', split='train')
test_set = load_dataset('sagnikrayc/mctest', 'mc160', split='test')
validation_set = load_dataset('sagnikrayc/mctest', 'mc160', split='validation')

In [3]:
#take a look at the a sample of the dataset
print(train_set)
# take a look at the first answer, answer options
print('---story---')
print(train_set[0]['story'])
print('---question---')
print(train_set[0]['question'])
print('---answer options---')
print(train_set[0]['answer_options'])
print('---answer---')
print(train_set[0]['answer'])

Dataset({
    features: ['idx', 'question', 'story', 'properties', 'answer_options', 'answer', 'question_is_multiple'],
    num_rows: 280
})
---story---
Tom had to fix some things around the house. He had to fix the door. He had to fix the\newlinewindow. But before he did anything he had to fix the toilet. Tom called over his best\newlinefriend Jim to help him. Jim brought with him his friends Molly and Holly. Tom thought that\newlineJim was going to bring Dolly with him but he didn't. The four of them got to work right\newlineaway. Fixing the toilet was easy. Fixing the door was also easy but fixing the window was\newlinevery hard. The window was stuck and could not be opened. They all pushed on the window\newlinereally hard until finally it opened. Once the window was fixed the four of them made a\newlinedelicious dinner and talked about all of the good work that they had done. Tom was glad\newlinethat he had such good friends to help him with his work.
---question---
What was the ha

In [4]:
# Global variables for prompt
PREFIX = '''
###### Instructions ######
Read the following story and the multiple-choice question, analyze step by step, select the correct option, and give the option letter (e.g., A or B) as your answer.
Use the following format to provide your answer and confidence level:
Explanation: [insert step-by-step analysis here]
Answer and Confidence (0-100): [Your answer, e.g., B], [Your confidence level, e.g., 80]%
Note: The confidence level indicates how certain you are about your answer, expressed as a percentage.
'''
openai.api_key = OPENAI_API_KEY

In [5]:
def get_last_processed_idx(checkpoint_file):
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as file:
            last_idx = file.readline()
            return int(last_idx.strip()) if last_idx else 0
    else:
        return 0
    
def set_checkpoint_idx(checkpoint_file, idx):
    with open(checkpoint_file, 'w') as file:
        file.write(str(idx))

In [6]:
def process_dataset(dataset, csv_file_path, checkpoint_file):
    start_idx = get_last_processed_idx(checkpoint_file)
    for idx in tqdm(range(start_idx, len(dataset))):
        try:
            question = dataset['question'][idx]
            story = dataset['story'][idx]
            options = dataset['answer_options'][idx]
            answer = dataset['answer'][idx]

            formatted_options = [f"{key}: {value}" for key, value in options.items()]
            question_input = f"###### Story ######\n{story}\n\n###### Question ######\n{question}\n" + "\n".join(formatted_options)
            prompt = PREFIX + f"{question_input}"

            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a chatbot trained to answer multiple-choice questions."},
                    {"role": "user", "content": prompt},
                ]
            )

            output = response['choices'][0]['message']['content'].strip()

            explanation_match = re.search(r'Explanation: (.*)\n', output)
            explanation = explanation_match.group(1) if explanation_match else "No explanation found."

            answer_confidence_match = re.search(r'Answer and Confidence \((0-100)\): ([A-D]), (\d+)%', output)
            predicted_answer = answer_confidence_match.group(2).strip() if answer_confidence_match else "No answer found."
            confidence_level = int(answer_confidence_match.group(3)) if answer_confidence_match else "No confidence level found."

            with open(csv_file_path, 'a+', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                if os.path.getsize(csv_file_path) == 0:
                    writer.writerow(["IDX", "Question", "Story", "Options", "Predicted Answer", "Actual Answer", "Confidence Level", "Explanation"])
                writer.writerow([idx, question, story, "\n".join(formatted_options), predicted_answer, answer, confidence_level, explanation])

            set_checkpoint_idx(checkpoint_file, idx + 1)

        except Exception as e:
            print(f"An error occurred at index {idx}: {e}")
            # If you want to stop on error use 'break', to continue to the next record use 'continue'
            # break
            continue

In [3]:
train_csv_output_path = 'mctest160_train_COT.csv'
test_csv_output_path = 'mctest160_test_COT.csv'
validation_csv_output_path = 'mctest160_validation_COT.csv'
train_checkpoint_file = 'mctest160_train_checkpoint.txt'
test_checkpoint_file = 'mctest160_test_checkpoint.txt'
validation_checkpoint_file = 'mctest160_validation_checkpoint.txt'

In [8]:
process_dataset(train_set, train_csv_output_path, train_checkpoint_file)

100%|██████████| 41/41 [04:59<00:00,  7.30s/it]


In [9]:
process_dataset(test_set, test_csv_output_path, test_checkpoint_file)

100%|██████████| 240/240 [29:19<00:00,  7.33s/it]


In [10]:
process_dataset(validation_set, validation_csv_output_path, validation_checkpoint_file)

100%|██████████| 120/120 [15:06<00:00,  7.56s/it]


# Prepare for Hugging Face Dataset

In [6]:
# put data to jsonl format
import jsonlines
import json

def convert_to_jsonl(csv_file_path, jsonl_file_path):
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        with jsonlines.open(jsonl_file_path, mode='w') as writer:
            for row in csv_reader:
                writer.write(row)

train_jsonl_output_path = 'train.jsonl'
test_jsonl_output_path = 'test.jsonl'
validation_jsonl_output_path = 'validation.jsonl'

convert_to_jsonl(train_csv_output_path, train_jsonl_output_path)
convert_to_jsonl(test_csv_output_path, test_jsonl_output_path)
convert_to_jsonl(validation_csv_output_path, validation_jsonl_output_path)

# Test to import from Hugging Face

In [7]:
mc160_train = load_dataset('BENBENBENb/mc160cot', split='train')
mc160_test = load_dataset('BENBENBENb/mc160cot', split='test')
mc160_validation = load_dataset('BENBENBENb/mc160cot', split='validation')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]